# Описание проекта


Построена модель, которая по данным с параметрами добычи и очистки предсказывает коэффициент восстановления золота из золотосодержащей руды. 

Задачи:

- Подготовить данные;
- Провести исследовательский анализ данных;
- Построить и обучить модель.

## Подготовка данных

### Импорт библиотек

In [1]:
import pandas as pd                 
import numpy as np
import matplotlib.pyplot as plt
import math                                          # импорт библиотеки math
from sklearn.metrics import mean_absolute_error      # импорт метода вычисления МАЕ
from scipy import stats as st
from sklearn.model_selection import train_test_split, cross_val_predict, cross_validate # импорт методов разбиения на выборки
# и кросс-валидации
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

In [1]:
train = pd.read_csv('...')   # чтение обучающей выборки
test = pd.read_csv('...')     # чтение тестовой выборки
data = pd.read_csv('...')     # чтение исходных данных

In [ ]:
test.info()                                          # вывод информации об обучающей выборке

**Вывод**

Тестовая выборка содержит незначительное число пропусков.

In [ ]:
data['rougher.input.feed_size'].to_csv('Sorting')                                   # вывод информации об обучающей выборке

In [ ]:
file = pd.read_csv('Sorting')
display(file)

**Вывод**

Обучающая выборка содержит параметры с большим числом пропусков, например secondary_cleaner.output.tail_sol и др.

In [ ]:
data.info()                             # вывод информации об исходных данных

**Вывод**

Исходные данные тоже содержат параметры, в которых много пропусков.

Кроме того, дата во всех датафреймах имеет тип object, что не есть хорошо, необходимо исправить тип данных на datetime. Также обнаружено, что датафрейм с тестовой выборкой содержит усечённое число признаков.

### Приведение данных к нужному типу

In [ ]:
train['date'] = pd.to_datetime(                           #
    train['date'], format='%Y-%m-%d %H:%M:%S')            #
data['date'] = pd.to_datetime(                            #    приведение признака 'date' к формату datetime
    data['date'], format='%Y-%m-%d %H:%M:%S')             #               во всех трёх датафреймах
test['date'] = pd.to_datetime(                            #
    test['date'], format='%Y-%m-%d %H:%M:%S')             #
train.info(verbose=False)                                 #

### Проверка правильности указанной эффективности

In [ ]:
recovery = train['rougher.output.concentrate_au'] * (train['rougher.input.feed_au'] -                              # расчёт эфф-
                                                    train['rougher.output.tail_au']) / (                           # ективности
train['rougher.input.feed_au'] * (train['rougher.output.concentrate_au'] - train['rougher.output.tail_au'])) * 100 # флотации

MAE = mean_absolute_error(train['rougher.output.recovery'], recovery)        # МАЕ между посчитанной и указанной эффективности
print(f"МАЕ указанной и посчитанной эффективности после флотации: {MAE}")

**Вывод**

Указанная эффективность рассчитана верно, ведь МАЕ практически 0.

### Признаки, отсутствующие в тестовой выборке

In [ ]:
no_columns = pd.Series(test.columns.symmetric_difference(train.columns))  # список отсутствующих колонок в тестовой выборке
train[no_columns].info()                                                  # признаки в этих колонках в обучающей выборке

**Вывод**

Видно, что в тестовой выборке отсутствует много признаков, в том числе и целевой rougher.output.recovery. Все эти данные относятся к типу float. Необходимо удалить из обучающей выборки эти признаки, чтобы тест проходил правильно. Целевые признаки будут взяты из исходных данных.

Добавим целевые признаки в тестовую выборку.

In [ ]:
test_date = test['date']              # создадим переменную с датами из тестовой выборки
test['rougher.output.recovery'] = data.query('date in @test_date')['rougher.output.recovery'].reset_index(drop = True) # срез из
# исходных данных по датам из тестовой выборки и признаку эффективности флотации добавляем в тестовую выборку
test['final.output.recovery'] = data.query('date in @test_date')['final.output.recovery'].reset_index(drop = True) # срез из
# исходных данных по датам из тестовой выборки и признаку финальной эффективности добавляем в тестовую выборку
test.head()

Удалим из обучающей вборки признаки, отсутствующие в тестовой для правильного теста будущей модели.

In [ ]:
no_columns = pd.Series(test.columns.symmetric_difference(train.columns)) # список отсутствующих колонок в тестовой выборке
train = train.drop(no_columns, axis = 1)            # удаление этих колонок в обучающей выборке
print(f'Размеры обучающей выборки: {train.shape}') 
print(f'Размеры тестовой выборки: {test.shape}')


**Вывод**

Теперь обучающая и тестовая выборки имеют одинаковое число признаков.

### Заполнение пропусков

Заполнять пропуски будем исходя из того факта, что измерения, проведённые в близкие времена должны быть похожи, поэтому в датафрейме, отсортированном по дате пропуск можно восстановить по соседним значениям.


In [ ]:
test_train = pd.concat([test,train]).sort_values(by='date').reset_index() # объединение тестовой и обучающей выборки для запол
# нения пропусков, поскольку нам необходимы данные с последовательным измененением времени при движении по датафрейму
test_train.info(verbose=False)

In [ ]:
test_train = test_train.fillna(method='ffill') # заполняем пропуски предыдущим значением признака
test_train.info()

In [ ]:
test_date = test['date']         # создадим переменную с датами из тестовой выборки
train_date = train['date']       # создадим переменную с датами из обучающей выборки

test =  test_train.query('date in @test_date').reset_index(drop=True).drop('index',axis=1)  # делаем срез по тем датам, 
# которые есть в тестовой выборке
train = test_train.query('date in @train_date').reset_index(drop=True).drop('index',axis=1) # делаем срез по тем датам, 
# которые есть в обучающей выборке
display(test.head())
train.head()

## Анализ данных

In [ ]:
def concentration(metal):                                      # функция подсчёта средней концентрации
    concentr = []
    for i in  data.columns:                                    # цикл по всем признакам
        if 'concentrate_'+metal in i or 'feed_'+metal  in i:   # если признак содержит в названии концентрацию
                                                               # и связан с выбранным металлом, то 
            concentr.append(data[i].mean())        # считаем среднюю концентрацию 
            
    
    a = pd.Series(concentr, index=[3,2,0,1]).sort_index() # располагаем концентрации в порядке технологического процесса
    a.index = ['Сырьё','Черновой конц.','Конц. после 1 очистки', 'Конц. после 2 очистки']

    return a

plt.title("Золото")
concentration('au').plot(kind='bar').set_ylabel("Концентрация")
plt.show()
plt.title("Серебро")
concentration('ag').plot(kind='bar').set_ylabel("Концентрация")
plt.show()
plt.title("Свинец")
concentration('pb').plot(kind='bar').set_ylabel("Концентрация")
plt.show()

**Вывод**

По мере прохождения технологических циклов концентрация золота увеличивается, серебра - меняется не монотонно, свинца - растёт.

### Проверка распределений гранул сырья

In [ ]:
train['rougher.input.feed_size'].hist(bins=100, range = (0,200),  density=True) # распределение гранул сырья в обучающей выборке
test['rougher.input.feed_size'].hist(bins=100, range = (0,200), density=True) # распределение гранул сырья в тестовой выборке

Выдвигаем гипотезу H0, что распределения похожи. H1 - распределения отличаются.

In [ ]:
alpha = 0.05  # пороговое значение устанавливаем на 5% 

feed_size_train = train['rougher.input.feed_size']       
feed_size_test = test['rougher.input.feed_size']

results = st.ks_2samp(feed_size_train,feed_size_test)     # критерий Колмогорова-Смирнова


print('p-значение: ', results.pvalue)                    # выводим p-value на экран

if results.pvalue < alpha:                               # сравниваем p-value с пороговым значением
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

**Вывод**

Подозрения о том, что распределения отличаются подтвердились и критерием Колмогорова-Смирнова. Поэтому правильней, чтобы обучение и проверка модели проходили на других выборках. Но поскольку наша задача, всё-таки, использовать уже готовые выборки, придётся смириться с грядущей ошибкой в оценке.

## Исследование поведения суммарной концентрации всех веществ

Поскольку в тестовой выборке представлены не все параметры, которые нужны для расчёта суммарной концентрации, то считать её мы будем из исходных данных.

In [ ]:
sum_concentration_1 = data['rougher.input.feed_ag'] + data['rougher.input.feed_au'] + \
data['rougher.input.feed_pb'] + data['rougher.input.feed_sol']               # расчёт суммарной концентрации в сырье
sum_concentration_1.hist(bins=100, range = (0,100))                                              # построение гистограммы
print(f'Средняя концентрация всех веществ сырья {sum_concentration_1.mean()}')                   


sum_concentration_2 = data['rougher.output.concentrate_ag'] + data['rougher.output.concentrate_ag'] + \
data['rougher.output.concentrate_ag'] + data['rougher.output.concentrate_ag']         # расчёт суммарной концентрации в 
# черновом концентрате
sum_concentration_2.hist(bins=100, range = (0,100))
print(f'Средняя концентрация всех веществ чернового концентрата {sum_concentration_2.mean()}')

sum_concentration = data['final.output.concentrate_ag'] + data['final.output.concentrate_au'] + \
data['final.output.concentrate_pb'] + data['final.output.concentrate_sol']   # расчёт суммарной концентрации в финальном 
# концентрате
sum_concentration.hist(bins=100, range = (0,100))
plt.legend(['Сырьё', 'Черновой концентрат','Готовый концентрат'])                               # создание легенды
print(f'Средняя концентрация всех веществ готового концентрата {sum_concentration.mean()}')

data['sum_concentration_feed'] = sum_concentration_1                #
data['sum_concentration_rougher'] = sum_concentration_2             #   добавление этих новых метрик в исходные данные
data['sum_concentration_final'] = sum_concentration                 #



**Вывод**

Из графика видно, что распределения концентраций всех веществ по трём стадиям сильно различаются. Самый узкий пик (самый малый разброс) имеет распределение концентрации готового сырья. Самый большой разброс имеет распределение концентраций чернового концентрата. Средняя концентрация веществ после флотации падает, а после двух этапов очистки - растёт.

В распределениях обнаружены артефакты в виде длинных хвостов, соответствующих малым значениям концентрации или вообще нулевым значениям. Такие данные нужно удалить из рассмотрения. Границей адекватного значения концетрации было выбрано значение 20%, поскольку большая часть всех распределений лежит правее этой цифры. 

### Удаление артефактов

Вначале, удалим лишние данные из исходных данных.

In [ ]:
data = data.query('sum_concentration_feed > 20 and sum_concentration_rougher > 20 and sum_concentration_final > 20') # убираем
# данные в исходных данных, где параметры суммарной концентрации явно занижены
data['sum_concentration_feed'].hist(bins=100, range = (0,100))       #
data['sum_concentration_rougher'].hist(bins=100, range = (0,100))    # проверяем обрезку
data['sum_concentration_final'].hist(bins=100, range = (0,100))      #

In [ ]:
good_date = data['date']      # создадим переменную, в которую положим дату объектов после обрезки
test_before = test
train_before = train
test = test.query('date in @good_date').reset_index(drop=True)   # выберем в тестовой выборке только те
# объекты, которые присутствуют в исходных данных после удаления артефактов
train = train.query('date in @good_date').reset_index(drop=True) # выберем в обучающей выборке только те
# объекты, которые присутствуют в исходных данных после удаления артефактов
print(f'Размеры тестовой выборки до обрезки:{test_before.shape} и после: {test.shape}')
print(f'Размеры обучающей выборки до обрезки: {train.shape} и после: {train.shape}')


**Вывод**

Все объекты с сильно заниженными значениями суммарной концентрации попали в тестовую выборку. Мы их из рассмотрения убрали.

## Модель

### Функция для расчёта sMAPE

Из определения метрики видно, что в случае равенства 0 предсказанной величины и целевого признака возникает неопределённость 0/0. Это необходимо учитывать в расчёте.

In [ ]:
def sMAPE(predictions,target):
    for i in predictions.index:                          # цикл по всем предсказаниям
        if predictions[i] == 0 and target[i] == 0:       # если возникает неопределённость 0/0, 
            predictions = predictions.drop(i)            # то убираем объект из рассмотрения
            target = target.drop(i)                      #
    quality = 1/len(predictions) * sum(abs(predictions-target)/((abs(predictions)+abs(target))/2))*100  # вычисдяем величину
# sMAPE по указанной формуле
    return quality
    

In [ ]:
features = train.drop(['date','final.output.recovery','rougher.output.recovery'], axis=1) # выделяем признаки
target_rougher = train['rougher.output.recovery']                # выделяем первый целевой признак

model_1 = LinearRegression()

predictions = pd.Series(cross_val_predict(model_1, features, target_rougher, cv=10), index=target_rougher.index) # получаем 
# предсказания после кросс-валидации по 10 блокам


print(f"sMAPE прогноза эффективности обогащения чернового концентрата в модели лин. регрессии: \
{sMAPE(predictions, target_rougher)} %")

metrics_best = 100                         # делаем начальное значение метрики наихудшим       
for depth in range(1,8):                   # цикл по глубине модели
    model_2 = DecisionTreeRegressor(random_state=12345, max_depth=depth)    # модель решающего дерева 
    predictions = pd.Series(cross_val_predict(model_2, features, target_rougher, cv=10), index=target_rougher.index) # получаем 
# предсказания после кросс-валидации по 10 блокам
    metrics = sMAPE(predictions, target_rougher)        # применение функции sMAPE и получение соотв. метрики
    if metrics < metrics_best:      #
        metrics_best = metrics      # поиск в теле цикла лучшей метрики и лучшего гиперпараметра глубины
        best_depth = depth          #
print(f"sMAPE прогноза эффективности обогащения чернового концентрата в модели решающего дерева: {metrics_best} %")
print(f'Лучшая глубина решающего дерева {best_depth}')

metrics_best = 100                         # делаем начальное значение метрики наихудшим
for depth in range(1,8):
    model_3 = RandomForestRegressor(random_state=12345, max_depth=depth)
    predictions = pd.Series(cross_val_predict(model_3, features, target_rougher, cv=2), index=target_rougher.index) # получаем 
# предсказания после кросс-валидации по 2 блокам
    metrics = sMAPE(predictions, target_rougher)       # применение функции sMAPE и получение соотв. метрики
    if metrics < metrics_best:      #
        metrics_best = metrics      # поиск в теле цикла лучшей метрики и лучшего гиперпараметра глубины
        best_depth = depth          #

print(f"sMAPE прогноза эффективности обогащения чернового концентрата в модели случайного леса: {metrics_best} %")
print(f'Лучшая глубина случайного леса: {best_depth}')

In [ ]:
target_final = train['final.output.recovery']                # выделяем второй целевой признак

predictions = pd.Series(cross_val_predict(model_1, features, target_final, cv=10), index=target_final.index) # получаем 
# предсказания после кросс-валидации по 10 блокам
print(f"sMAPE прогноза эффективности обогащения финального концентрата в модели лин. регрессии: {sMAPE(predictions, target_final)} %")

metrics_best = 100                 # делаем начальное значение метрики наихудшим 
for depth in range(1,8):
    model_2 = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    predictions = pd.Series(cross_val_predict(model_2, features, target_final, cv=10), index=target_final.index) # получаем 
# предсказания после кросс-валидации по 10 блокам
    metrics = sMAPE(predictions, target_final)     # применение функции sMAPE и получение соотв. метрики
    if metrics < metrics_best:      #
        metrics_best = metrics      # поиск в теле цикла лучшей метрики и лучшего гиперпараметра глубины
        best_depth = depth          #
print(f"sMAPE прогноза эффективности обогащения финального концентрата в модели решающего дерева: {metrics_best} %")
print(f'Лучшая глубина решающего дерева {best_depth}')

metrics_best = 100                  # делаем начальное значение метрики наихудшим
for depth in range(1,6):
    model_3 = RandomForestRegressor(random_state=12345, max_depth=depth)
    predictions = pd.Series(cross_val_predict(model_3, features, target_final, cv=2), index=target_final.index) # получаем 
# предсказания после кросс-валидации по 2 блокам
    metrics = sMAPE(predictions, target_rougher)  # применение функции sMAPE и получение соотв. метрики
    if metrics < metrics_best:      #
        metrics_best = metrics      # поиск в теле цикла лучшей метрики и лучшего гиперпараметра глубины
        best_depth = depth          #
print(f"sMAPE прогноза эффективности обогащения финального концентрата в модели случайного леса: {sMAPE(predictions, target_final)} %")
print(f'Лучшая глубина случайного леса: {best_depth}')

**Вывод**

В случае с прогнозом эффективности обогащения чернового концентрата лучшую метрику показывет модель линейной регрессии. Если же говорить о прогнозе эффективности обогащения финального концентрата, то здесь победителем стало решающее дерево с глубиной 5.

### Обучение лучших моделей

In [ ]:
best_model_1 = LinearRegression()
best_model_1.fit(features, target_rougher)  # обучение на всей обучающей выборке с первым целевым признаком

best_model_2 = DecisionTreeRegressor(random_state=12345, max_depth=5)  # модель решающего дерева с глубиной 5
best_model_2.fit(features, target_final)    # обучение на всей обучающей выборке со вторым целевым признаком

### Тестирование модели и подсчёт итоговой метрики

In [ ]:
test_null =  test.dropna()

features_test = test_null.drop(['date','final.output.recovery','rougher.output.recovery'], axis=1) # выделяем признаки
target_test = test_null['rougher.output.recovery']                # выделяем первый целевой признак


predictions = pd.Series(best_model_1.predict(features_test), index = target_test.index)
sMAPE_rougher = sMAPE(predictions, target_test)                   # применение функции sMAPE и получение соотв. метрики



target_test = test_null['final.output.recovery']                  # выделяем второй целевой признак


predictions = pd.Series(best_model_2.predict(features_test), index = target_test.index)
sMAPE_final = sMAPE(predictions, target_test)                     # применение функции sMAPE и получение соотв. метрики
result = 0.25 * sMAPE_rougher + 0.75 * sMAPE_final                # расчёт sMAPE по указанной формуле
print(f'Итоговая sMAPE: {result} %')


target_test = test_null['rougher.output.recovery']                # выделяем первый целевой признак
dummy_regr_1 = DummyRegressor()
dummy_regr_1.fit(features, target_rougher)
predictions_dummy_1 = pd.Series(dummy_regr_1.predict(features_test), index = target_test.index)
sMAPE_rougher = sMAPE(predictions_dummy_1, target_test)

target_test = test_null['final.output.recovery']                  # выделяем второй целевой признак
dummy_regr_2 = DummyRegressor()
dummy_regr_2.fit(features, target_final)
predictions_dummy_2 = pd.Series(dummy_regr_2.predict(features_test), index = target_test.index)
sMAPE_final = sMAPE(predictions_dummy_2, target_test)
result_dummy = 0.25 * sMAPE_rougher + 0.75 * sMAPE_final                # расчёт sMAPE по указанной формуле
print(f'Для dummy-модели итоговая sMAPE: {result_dummy} %')


## Итоговый вывод

В ходе подготовки данных к построению модели была преобразована дата к нужному типу данных, из обучающей выборки удалены признаки, не встречающиеся в тестовой. Благодаря условию, что признаки двух объектов, полученных за близкое время, имеют близкие значения, был реализован метод восстановления пропусков: при встрече с пропуском берётся значение признака прошлого объекта.

Анализ концетраций трёх металлов показал, что по мере прохождения технологического цикла, концентрация золота и свинца растёт, а концентрация серебра ведёт себя не монотонно.

Проверка распределения размеров гранул сырья в тестовой и обучающей выборке с помощью применения критерия Колмогорова-Смирнова завершилась выводом: разбиение исходных данных было выполнено не оптимально, распределения отличаются (p-значение:  5.2e-213). Но сама форма распределений похожа. Конечно же, верней было бы провести переразбивку на две выборки, но у нас была задача отработать предсказания на конкретной выборке. 

Расчёт суммарной концентрации всех веществ на трёх этапах очистки и построение соответсвтующих распределений показали наличия артефактов: нулевых или очень малых значений суммарной концентрации (< 20%). От объектов с такими значениями нам пришлось избавиться. 

Для расчёта нашей метрики sMAPE была написана соответствующая функция, учитывающая возможность возникновения неопределённости 0/0.


После проведения кросс-валидации для трёх различных моделей, оказалось, что для предсказаний двух метрик оптимальными оказались две разные модели. Для эффективности обогащения чернового концентрата - линейная регрессия (sMAPE = 6.0556 %), а для финального концентрата - решающее дерево с глубиной 5 (sMAPE = 8.695 %)

Итоговое значение метрики sMAPE составило: 7.442 %

## Чек-лист готовности проекта

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке выполнения
- [x]  Выполнен шаг 1: данные подготовлены
    - [x]  Проверена формула вычисления эффективности обогащения
    - [x]  Проанализированы признаки, недоступные в тестовой выборке
    - [x]  Проведена предобработка данных
- [x]  Выполнен шаг 2: данные проанализированы
    - [x]  Исследовано изменение концентрации элементов на каждом этапе
    - [x]  Проанализированы распределения размеров гранул на обучающей и тестовой выборках
    - [x]  Исследованы суммарные концентрации
- [x]  Выполнен шаг 3: построена модель прогнозирования
    - [x]  Написана функция для вычисления итогового *sMAPE*
    - [x]  Обучено и проверено несколько моделей
    - [x]  Выбрана лучшая модель, её качество проверено на тестовой выборке